# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 12
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiDi

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 60
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiD

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355101859059                   -1.10    6.0    152ms
  2   -9.356737913282       -2.79       -1.43    1.0   76.1ms
  3   -9.357057630142       -3.50       -2.77    2.4    144ms
  4   -9.357110188977       -4.28       -2.99    8.0    498ms
  5   -9.357110793981       -6.22       -3.13    1.1   77.7ms
  6   -9.357111151947       -6.45       -3.28    1.1   77.0ms
  7   -9.357111369655       -6.66       -3.44    1.0   78.4ms
  8   -9.357111458165       -7.05       -3.61    1.0   76.1ms
  9   -9.357111498465       -7.39       -3.89    2.1   91.1ms
 10   -9.357111507504       -8.04       -4.23    1.5   88.6ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.75236958959                   -0.97    6.9    478ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     6
│     1
│     2
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.75920409423       -2.17       -1.33    1.8    268ms
  3   -18.79242271442       -1.48       -2.20    4.0    318ms
  4   -18.79259658569       -3.76       -2.53    7.0    483ms
  5   -18.79260434958       -5.11       -3.22    1.6    264ms
  6   -18.79260624130       -5.72       -3.61    5.4    410ms
  7   -18.79260638310       -6.85       -3.99    3.5    300ms
  8   -18.79260639393       -7.97       -4.48    2.8    291ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55030338104                   -0.84    9.0    1.91s
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      1
│      1
│      1
│      1
│      1
│     11
│      1
│      4
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -37.55751817870       -2.14       -1.23    2.6    756ms
  3   -37.55991785175       -2.62       -2.44    6.4    1.14s
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     15
│      5
│     20
│     24
│     26
│     19
│     22
│     12
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  4   -37.56493629018       -2.30       -2.17   17.9    2.14s
  5   -37.56493543168   +   -6.07       -2.12    1.6    827ms
  6   -37.56494919346       -4.86       -3.60    1.0    765ms
  7   -37.56494965467       -6.34       -3.75   12.2    1.88s
  8

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355219050565                   -1.10    5.9    169ms
  2   -9.356820268352       -2.80       -1.91    1.0   65.9ms
  3   -9.357087174574       -3.57       -2.57    3.4    116ms
  4   -9.357090600285       -5.47       -2.59    5.6    148ms
  5   -9.357111153629       -4.69       -3.58    1.0   67.0ms
  6   -9.357111417630       -6.58       -3.85    3.8    118ms
  7   -9.357111502063       -7.07       -4.36    1.5   74.2ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54832379542                   -0.84    9.1    1.74s
  2   -37.53629195692   +   -1.92       -1.56    2.6    671ms
  3   -18.05145641754   +    1.29       -0.42    9.1    1.69s
  4   -37.50877729934        1.29       -1.56    7.5    1.55s
  5   -37.52939138192       -1.69       -1.72    2.4    845ms
  6   -37.39334843298   +   -0.87       -1.44    5.2    1.34s
  7   -37.52660067852       -0.88       -1.75    4.5    1.04s
  8   -37.56377191234       -1.43       -2.41    1.8    772ms
  9   -37.56483008783       -2.98       -2.65    4.5    1.27s
 10   -37.56443165149   +   -3.40       -2.47    2.9    1.05s
 11   -37.56486152072       -3.37       -2.91    2.6    1.21s
 12   -37.56493334126       -4.14       -3.33    2.0    795ms
 13   -37.56494662797       -4.88       -3.63    2.9    934ms
 14   -37.56494954135       -5.54       -3.90    3.4    1.07s
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.